# Local Level Hainich
> simple local level model on Hainich data 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import altair as alt
alt.renderers.enable('mimetype')

RendererRegistry.enable('mimetype')

In [ ]:
from meteo_imp.data import hai, units
from meteo_imp.data_preparation import MeteoDataTest
from meteo_imp.kalman.imputation import KalmanImputation
from meteo_imp.kalman.model import *
from ipywidgets import interact, interact_manual, IntSlider

In [ ]:
hai

,TA,SW_IN,VPD
TIMESTAMP_END,,,
2000-01-01 00:30:00,-0.60,0.0,0.222
2000-01-01 01:00:00,-0.65,0.0,0.122
2000-01-01 01:30:00,-0.58,0.0,0.090
2000-01-01 02:00:00,-0.51,0.0,0.110
2000-01-01 02:30:00,-0.49,0.0,0.102
...,...,...,...
2000-01-05 02:00:00,4.74,0.0,1.191
2000-01-05 02:30:00,4.75,0.0,1.057
2000-01-05 03:00:00,4.76,0.0,0.935


In [ ]:
data = MeteoDataTest(hai).add_gap(10, ['TA', 'SW_IN', 'VPD'], 30)

In [ ]:
data.data

,TA,SW_IN,VPD
TIMESTAMP_END,,,
2000-01-01 00:30:00,-0.60,0.0,0.222
2000-01-01 01:00:00,-0.65,0.0,0.122
2000-01-01 01:30:00,-0.58,0.0,0.090
2000-01-01 02:00:00,-0.51,0.0,0.110
2000-01-01 02:30:00,-0.49,0.0,0.102
...,...,...,...
2000-01-05 02:00:00,4.74,0.0,1.191
2000-01-05 02:30:00,4.75,0.0,1.057
2000-01-05 03:00:00,4.76,0.0,0.935


**Warning WIP. Data is not standardized (at it should)**

In [ ]:
imp = KalmanImputation(data.data, LocalLevelModel)

In [ ]:
imp.fit()

<meteo_imp.kalman.imputation.KalmanImputation>

In [ ]:
imp.impute()

TypeError: KalmanModel.predict() takes 2 positional arguments but 3 were given

In [ ]:
data.data.columns

In [ ]:
res = imp.to_result(data.data_compl_tidy, var_names= data.data.columns, units=units, pred_all=True)

In [ ]:
res.display_results()

## Interactive

In [ ]:
def gap2res(var_sel, gap_len, gap_start, model, n_iter):
    data = MeteoDataTest(hai).add_gap(gap_len, var_sel, gap_start)
    return KalmanImputation(data.data, model).fit(n_iter=n_iter).to_result(data.data_compl_tidy, var_names= data.data.columns, units=units, pred_all=True)

In [ ]:
var_sel = data.data.columns

In [ ]:
%time gap2res(var_sel, 10, 10, LocalLevelModel, 5)

In [ ]:
gaps = [2, 5, 7, 10, 20, 30, 50, 100]
gap_starts = [0, 30, 60, 90]

In [ ]:
#| include: false
@interact_manual(TA=True, SW_IN=True, VPD=True,
          gap_len=IntSlider(10, 1,100),
          gap_start=IntSlider(30, 1,100),
          model=[LocalLevelModel, LocalSlopeModel],
          n_iter = 5)
def show_diff_gaps_res(TA, SW_IN, VPD, gap_len, gap_start, model, n_iter):
    var_sel = []
    if TA: var_sel.append('TA')
    if SW_IN: var_sel.append('SW_IN')
    if VPD: var_sel.append('VPD')
    var_sel = (*var_sel,)
    gap2res(var_sel, gap_len, gap_start, model, n_iter).display_results()
    

## Local Slope Model 

In [ ]:
gap_len = 30
gap_start = 40
var_sel = ('TA', 'SW_IN', 'VPD')
n_iter = 10

In [ ]:
data = MeteoDataTest(hai).add_gap(gap_len, var_sel, gap_start)
imp_ls = KalmanImputation(data.data, LocalSlopeModel).fit(n_iter=n_iter)
res_ls = imp_ls.to_result(data.data_compl_tidy, var_names= data.data.columns, units=units, pred_all=True)

In [ ]:
res_ls.display_results()

In [ ]:
imp

In [ ]:
data = MeteoDataTest(hai).add_gap(gap_len, var_sel, gap_start)
imp_ls = KalmanImputation(data.data, LocalSlopeModel).fit(n_iter=n_iter, smooth=False).to_result(data.data_compl_tidy, var_names= data.data.columns, units=units, pred_all=True)

In [ ]:
imp_ls.display_results()